一、比赛背景

本次比赛是Datawhale与天池联合发起的入门赛，题目是：零基础入门金融风控之贷款违约预测挑战赛。 赛题以金融风控中的个人信贷为背景，要求选手根据贷款申请人的数据信息预测其是否有违约的可能，以此判断是否通过此项贷款，这是一个典型的分类问题。

二、赛题数据

赛题以预测用户贷款是否违约为任务，数据来自某信贷平台的贷款记录，总数据量超过120w，包含47列变量信息，其中15列为匿名变量。为了保证比赛的公平性，将会从中抽取80万条作为训练集，20万条作为测试集A，20万条作为测试集B，同时会对employmentTitle、purpose、postCode和title等信息进行脱敏。

train.csv

Field                                      Description
id	                                为贷款清单分配的唯一信用证标识
loanAmnt	                   贷款金额
term	                          贷款期限（year）
interestRate	               贷款利率
installment	                    分期付款金额
grade	                         贷款等级
subGrade	                  贷款等级之子级
employmentTitle	            就业职称
employmentLength	     就业年限（年）
homeOwnership	          借款人在登记时提供的房屋所有权状况
annualIncome	            年收入
verificationStatus	         验证状态
issueDate	                  贷款发放的月份
purpose	                       借款人在贷款申请时的贷款用途类别
postCode	                  借款人在贷款申请中提供的邮政编码的前3位数字
regionCode	                 地区编码
dti	                               债务收入比
delinquency_2years	      借款人过去2年信用档案中逾期30天以上的违约事件数
ficoRangeLow	            借款人在贷款发放时的fico所属的下限范围
ficoRangeHigh	            借款人在贷款发放时的fico所属的上限范围
openAcc	                      借款人信用档案中未结信用额度的数量
pubRec	                       贬损公共记录的数量
pubRecBankruptcies	    公开记录清除的数量
revolBal	                    信贷周转余额合计
revolUtil	                    循环额度利用率，或借款人使用的相对于所有可用循环信贷的信贷金额
totalAcc	                   借款人信用档案中当前的信用额度总数
initialListStatus	          贷款的初始列表状态
applicationType	           表明贷款是个人申请还是与两个共同借款人的联合申请
earliesCreditLine	        借款人最早报告的信用额度开立的月份
title	                          借款人提供的贷款名称
policyCode	                公开可用的策略_代码=1新产品不公开可用的策略_代码=2
n系列匿名特征	             匿名特征n0-n14，为一些贷款人行为计数特征的处理

In [ ]:
三、评测标准

提交结果为每个测试样本是1的概率，也就是y为1的概率。评价指标是：AUC（Area Under Curve）评估模型效果（越大越好)。

分类算法常见的评估指标如下：

1、准确率（Accuracy） 准确率是常用的一个评价指标，但是不适合样本不均衡的情况。
    Accuracy = (TP + TN) / (TP + TN + FP + FN)
    
    缺点：在正负样本不平衡的情况下，这个指标有很大的缺陷。例如：给定一组测试样本共1100个实例，其中1000个是正类，剩余100个是负类。即使分类模型将所有实例均预测为正类，Accuracy也有90%以上，这样就没什么意义了。

2、精确率（Precision）又称查准率，正确预测为正样本（TP）占预测为正样本(TP+FP)的百分比。
    Precision = (TP) / (TP + FP)

3、召回率（Recall) 又称为查全率，正确预测为正样本（TP）占正样本(TP+FN)的百分比。 
    Recall = (TP) / (TP + FN)
    
4、P-R曲线（Precision-Recall Curve)是描述精确率和召回率变化的曲线

5、F1 Score: 精确率、召回率的结合。定义如下：
    F1-Score = 2 * Precison * Recall / (Precision + Recall)
    
    我们当然希望检索结果（分类结果）的Precision越高越好，同时Recall也越高越好，但事实上这两者在某些情况下有矛盾的。
    比如极端情况下，我们只搜索出了一个结果，且是准确的（分类后的正确实例只有一个，且该实例原本就是正实例），那么Precision就是100%，但是Recall就会很低；
    而如果把所有结果都返回（所有的结果都被分类为正实例），那么Recall是100%，但是Precision就会很低。
    因此在不同的场合中需要自己判断希望Precision比较高还是Recall比较高。如果是做实验研究，可以绘制Precision-Recall曲线来帮助分析。

6、综合评价指标F-Measure

Precision和Recall指标有时候会出现矛盾的情况，这样就需要综合考虑他们，最常见的方法就是F-Measure（又称为F-Score)。
F-Score是Precision和Recall的加权调和平均：

F-Score= （1 + a**2）* Precision * Recall / (a**2 * Precision + Recall)

当参数a=1时，就是最常见的F1, 因此F1综合了P和R的结果，当F1较高时则能说明试验方法比较有效。

7、ROC（Receiver Operating Characteristic）

为什么引入ROC曲线？

Motivation1：在一个二分类模型中，对于所得到的连续结果，假设已确定一个阈值，比如说0.6，大于这个值的实例为正类，小于这个值则为负类。
如果减小阈值，比如减到0.5，固然能之别出更多的正类，也就是提高了识别出的正例占所有正例的比例，即TPR值变大；
但同时也将更多的负实例当作了正实例，即，提高了FPR。为了形象化这一变化，引入了ROC，ROC可以用于评价一个分类器。
Motivation2：在类不平衡的情况下，如正样本90个，负样本10个，直接把所有样本分类为正样本，得到识别率为90%。但这显然是没有意义的。
单纯根据Precision和Rcall来衡量算法的优劣已经不能表征这种病态问题

ROC空间将真阳率（TPR）定义为 Y 轴， 假阳率（FPR）定义为 X 轴。
TPR：在所有实际为正例的样本中，被正确地判断为正的比率。 TPR = (TP) / (TP + FN)
FPR：在所有实际为负例的样本中，被错误地判断为正的比率。 FPR = (FP) / (FP + TN)

ROC曲线有个很好的特性：当测试集中的正负样本的分布变换的时候，ROC曲线能够保持不变。
在实际的数据集中经常会出现样本类不平衡，即正负样本比例差距较大，而且测试数据中的正负样本也可能随着时间变化。

8、AUC(Area Under Curve) 被定义ROC曲线下的面积，显然这个面积的数值不会大于1。
    一个完美的分类器的AUC为1.0，而随机猜测的AUC为0.5，显然AUC值在0和1之间，并且数值越高，代表模型的性能越好。
    AUC的物理意义：首先AUC值是一个概率值。假设分类器的输出是样本属于正类的Score(置信度)，则AUC的物理意义为任取一对（正、负）样本，正样本的Score大于负样本的Score的概率。

In [ ]:
四、结果提交

提交前请确保预测结果的格式与sample_submit.csv中的格式一致，以及提交文件后缀名为csv。

形式如下：

id,isDefault
800000,0.5
800001,0.5
800002,0.5
800003,0.5